# Trabalho 02 SCC0650 - Computação Gráfica

**Autor** : Tarcídio Antônio Júnior - 10748347

**Introdução:**

A computação gráfica é uma disciplina da ciência da computação que lida com a geração, manipulação e representação de imagens e objetos visuais por meio de software e hardware. Ela desempenha um papel crucial em várias áreas, incluindo entretenimento, design, engenharia, medicina, educação e muitos outros campos. Neste trabalho, aborda-se algumas de suas bases.

**Objetivos do trabalho:**

Neste trabalho, busca-se sedimentar os seguintes conhecimentos: pipeline gráfico, API OpenGL, sistemas de janelas, primitivas geométricas, transformações geométricas 3D, coordenadas homogêneas, além de malhas e texturas. Para isso, o trabalho importa cinco objetos no formato wavefront (.obj) e suas respectivas texturas, permitindo que o usuário aplique transformações geométricas a partir do teclado.

**Descrição dos comandos:**

* Invocação: os objetos são apresentados de forma individual e acionados por eventos de teclado. Ao pressionar a tecla 1, o objeto 1 deve ser exibido, a tecla 2 para o objeto 2 e assim por diante. Todos partem da origem (0,0,0) e não ultrapassam o limite da janela. 
* Movimentação: as teclas 'a','d','s' e 'w' transladam o objeto para esquerda, direita, cima e baixo respectivamente
* Rotação: as teclas 'LEFT', 'RIGHT', 'UP' e 'DOWN' rotacionam o objeto. Os dois primeiros em relação ao eixo y e os dois últimos em relação ao eixo x. As teclas '+' e '-' rotacionam o objeto no eixo z.
* Escala: as teclas 'z' e 'x' aumenta e diminui respectivamente a escala do objeto
* Textura: a tecla 'p' ativa e desativa a textura
* Magnificação: a tecla 'v' altera entre as técnicas de magnificação linear e nearest

**Execução:**

Para este trabalho, é necessário que tenha instalado as seguintes bibliotecas: 
* glfw
* OpenGL
* numpy
* PIL

Para instalar as bibliotecas, pode-se utilizar os respectivos comandos:

```python
pip install glfw
pip install pyopengl
pip install numpy
pip install pillow
```

Para rodar o programa com o código principal no arquivo `.ipynb` (mais recomendado), basta executar a única célula disponível no arquivo `Trabalho01_SmallCode.ipynb`. `Trabalho01_BigCode.ipynb` é apenas uma versão não modularizada de `Trabalho01_SmallCode.ipynb`. Para executar na janela de comando do Windows ou Linux, basta executar 

```python
python3 main.py
```

**Ferramentas:**

Para criação deste projeto foi utilizado as seguintes ferramentas:
* Linguagem Python
* Sistemas de janela GLFW
* API OpenGL
* Bibliotecas: numpy, math e PIL

---

<sup>Instituto de Ciências Matemáticas e de Computação (ICMC) - Universidade de São Paulo (USP)</sup>


## Código pré loop principal

### Inicialização do glfw e criação da janela

In [797]:
# Bibliotecas
try: # Importa
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import glm
    from PIL import Image
except ImportError: # Caso não importe, instale
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install pyglm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import glm
    from PIL import Image

# Inicializando sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
# Entrada: porcentagem largura e altura da tela que deve ser ocupada
# Saída: tamanho da largura e altura da tela, além da posição x e y da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Trabalho 02"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
# Posicionando janela
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Criando shaders: Vertex e Fragment

In [798]:
# GLSL para Vertex Shader
vertex_code = """
        //POSIÇÃO VERTICES, TEXTURA E NORMAL
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;

        //VALORES REPASSADOS PARA O SHADER FRAGMENT
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
        
        //MATRIZES DE TRANSFORMACAO 
        uniform mat4 mat_pre_transl;
        uniform mat4 mat_rot_x;
        uniform mat4 mat_rot_y;
        uniform mat4 mat_rot_z; 
        uniform mat4 mat_scale;  
        uniform mat4 mat_transl;
        uniform mat4 view;
        uniform mat4 projection;   

        void main(){
            mat4 model = mat_transl * mat_scale * mat_rot_x * mat_rot_y * mat_rot_z * mat_pre_transl;
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = vec3(model *vec4(normals, 1.0));
        }
        """

# GLSL para Fragment Shader
fragment_code = """
        // POSICAO DA FONTE DE LUZ E COR DA LUZ
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // ILUMINACAO AMBIENTE E DIFUSA
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // ILUMINACAO ESPECULAR
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular

        // VARIAVEIS VINDOS DO SHADER VERTEX
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader

        // PARAMETRO DE TEXTURA
        uniform sampler2D samplerTexture;   

        // PARAMETRO DE GEOMETRIA
        uniform bool geometric;
        uniform vec4 color;
        
        void main(){
            // REFLEXAO AMBIENTE
            vec3 ambient = ka * lightColor;             
        
            // REFLEXAO DIFUSA
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // REFLEXAO ESPECULAR
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // MODELO DE PHONG
            vec4 texture;
            if(geometric)
                texture = color;
            else
                texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;
        }
        """

### Solicitando espaço, compilando e linkando shaders

In [799]:
# Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

# Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Tornando programa o atual
glUseProgram(program)

### Definindo classes para objetos do cenário

In [800]:
# graphic_element: classe para criação de elementos que se movimentam no cenário
# Esta classe é genérica, sendo utilizada principalmente para objetos criados pelo próprio programador
class graphic_element:
    def __init__(   self, 
                    inicial_vert = 0, num_vert = 0, 
                    pos_x = 0, pos_y = 0, pos_z = 0,
                    angle_x = 0, angle_y = 0, angle_z = 0,
                    scale = 1,
                    linear_speed = 0.05, angular_speed = 0.139, scale_speed = 0.2,
                    limit_sup = 100000, limit_inf = -1000000,
                    ka = 1, kd = 1, ks = 1, ns = 0.10,
                    light_source = False,
                    geometric = True,
                    ):
        # Atributos de identificação
        # Inicialmente estes atributos são inicializados com valores falsos, mas são alterados em seguida
        self._inicial_vert = inicial_vert   # Vértice inicial da onde deve partir a diretiva geométrica
        self._num_vert = num_vert           # Quantidade de vértices que o polígono tem
        # Mapa de eixo
        self.map_eixo = {}
        self.map_eixo['x'] = 0
        self.map_eixo['y'] = 1
        self.map_eixo['z'] = 2
        # Atributos de posição
        self._pos = [pos_x, pos_y, pos_z]   # Posição "central"
        self._linear_speed = linear_speed   # Velocidade do objeto para qualquer eixo
        self._limit_sup = limit_sup         # Limite superior dos dois eixos
        self._limit_inf = limit_inf         # Limite inferior dos dois eixos
        # Atributos de rotação
        self._angle = [angle_x, angle_y, angle_z]
        self._angular_speed = angular_speed
        self._cos = [math.cos(angle_x), math.cos(angle_y), math.cos(angle_z)]
        self._sin = [math.sin(angle_x), math.sin(angle_y), math.sin(angle_z)]
        # Atributos de escala
        self._scale = scale
        self._scale_speed = scale_speed
        # Indica quando o objeto está ativado ou não para ser desenhado
        self._on = True
        # Indica ativação ou não da textura do objeto
        self._polygonal_mode = GL_FILL
        # Parâmetros de iluminação
        self._ka = ka
        self._kd = kd
        self._ks = ks
        self._ns = ns
        # Se esse objeto é fonte de luz
        self._light_source = light_source
        # Se esse objeto é geometricou ou não 
        self._geometric = geometric

    # Ativa e desativa a textura do objeto
    def turn_texture(self):
        if self._polygonal_mode == GL_LINE:
            self._polygonal_mode = GL_FILL
        else:
            self._polygonal_mode = GL_LINE

    # Altera valores de idenficação para desenho
    # Entrada: 
        # Vértice inicial da onde deve partir a diretiva geométrica
        # Número de vértices do objeto
    def set_identification(self, inicial_vert, num_vert):
        self._inicial_vert = inicial_vert
        self._num_vert = num_vert
    
    # Calcula a validade da nova escala do objeto com base no limite de borda
    def _verify_limit(self,
                    new_value,
                    old_value,
                    pos = [False, False, False],
                    angle = [False, False, False],
                    scale = False):
        return new_value
    
    # Retorna estado de ativação do objeto
    def get_on(self):
        return self._on

    # Seta true para ativação do objeto
    def set_on(self):
        self._on = True
    
    # Seta off para ativação do objeto
    def set_off(self):
        self._on = False
    
    # Movimenta o objeto
    # Entrada: operação de somar ou subtrair, além do eixo da movimentação
    def move(self, operation, eixo):
        # A combinação da escolha 'somar','subtrair' e eixo gera todas as movimentações
        if self._on: # Se o objeto estiver ativo
            new_pos = old_pos = self._pos[self.map_eixo[eixo]]
            # Calcula a nova posição
            if operation == '+':
                new_pos += self._linear_speed
            elif operation == '-':
                new_pos -= self._linear_speed
            # Verifica se ela é válida e atribui
            pos_flag = [False, False, False]
            pos_flag[self.map_eixo[eixo]] = True
            if self._verify_limit(new_pos, old_pos, pos = pos_flag):
                self._pos[self.map_eixo[eixo]] = new_pos

    # Rotaciona o objeto
    # Entrada: operação de somar ou subtrair, além do eixo da rotação
    def rotate(self, operation, eixo):
        # A combinação da escolha 'somar','subtrair' e eixo gera todas as rotações
        if self._on:
            new_angle = old_angle = self._angle[self.map_eixo[eixo]]
            # Calcula o novo ângulo
            if operation == '+':
                new_angle += self._angular_speed
            elif operation == '-':
                new_angle -= self._angular_speed
            angle_flag = [False, False, False]
            angle_flag[self.map_eixo[eixo]] = True
            if self._verify_limit(new_angle, old_angle, angle = angle_flag):
                self._angle[self.map_eixo[eixo]] = new_angle
                self._cos[self.map_eixo[eixo]] = math.cos(new_angle)
                self._sin[self.map_eixo[eixo]] = math.sin(new_angle)

    # Escala o objeto
    # Entrada: operação de somar ou subtrair
    def distort(self, operation):
        if self._on:
            # Calcula a nova escala
            new_scale = old_scale = self._scale
            if operation == '+':
                new_scale += self._scale_speed
            elif operation == '-':
                new_scale -= self._scale_speed
            # Verifica se ela é válida e atribui
            if self._verify_limit(new_scale, old_scale, scale = True):
                self._scale = new_scale
    
    # Retorna a matriz de translação do objeto
    def _mat_translation(self):
        return np.array([   1.0, 0.0, 0.0, self._pos[0], 
                            0.0, 1.0, 0.0, self._pos[1], 
                            0.0, 0.0, 1.0, self._pos[2], 
                            0.0, 0.0, 0.0,        1.0], np.float32)
    
    # Retorna a matriz de escala do objeto
    def _mat_scale(self):
        return np.array([     self._scale,           0.0,           0.0, 0.0, 
                                      0.0,   self._scale,           0.0, 0.0, 
                                      0.0,           0.0,   self._scale, 0.0, 
                                      0.0,           0.0,           0.0, 1.0], np.float32)
    
    # Retorna a matriz de rotação no eixo z do objeto
    def _rotation_z(self):  
        return np.array([   self._cos[2], -self._sin[2], 0.0, 0.0, 
                            self._sin[2],  self._cos[2], 0.0, 0.0, 
                                     0.0,           0.0, 1.0, 0.0, 
                                     0.0,           0.0, 0.0, 1.0], np.float32)
    
    # Retorna a matriz de rotação no eixo x do objeto
    def _rotation_x(self): 
        return np.array([   1.0,          0.0,           0.0, 0.0, 
                            0.0, self._cos[0], -self._sin[0], 0.0, 
                            0.0, self._sin[0],  self._cos[0], 0.0, 
                            0.0,          0.0,           0.0, 1.0], np.float32)
    
    # Retorna a matriz de rotação no eixo y do objeto
    def _rotation_y(self):
        return np.array([    self._cos[1],  0.0, self._sin[1], 0.0, 
                                      0.0,  1.0,          0.0, 0.0, 
                            -self._sin[1],  0.0, self._cos[1], 0.0, 
                                      0.0,  0.0,          0.0, 1.0], np.float32)
    
    # Desenha o objeto na tela
    # Entrada: localizações dos qualificadores uniformes no shader relativos as matrizes de transformação geométrica
    def draw(self, loc_mat_pre_transl, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl,
             loc_ka, loc_kd, loc_ks, loc_ns, loc_light_pos, gl_Draw = GL_TRUE):
        if self._on:
            # Capturando qualificadores de matrizes de transformação
            loc_mat_rot_x = glGetUniformLocation(program, "mat_rot_x")
            loc_mat_rot_y = glGetUniformLocation(program, "mat_rot_y")
            loc_mat_rot_z = glGetUniformLocation(program, "mat_rot_z")
            loc_mat_scale = glGetUniformLocation(program, "mat_scale")
            loc_mat_transl = glGetUniformLocation(program, "mat_transl")
            loc_mat_pre_transl = glGetUniformLocation(program, "mat_pre_transl")

            # Capturado qualificadores de iluminação
            loc_ka = glGetUniformLocation(program, "ka")
            loc_kd = glGetUniformLocation(program, "kd") 
            loc_ks = glGetUniformLocation(program, "ks")
            loc_ns = glGetUniformLocation(program, "ns")
            loc_light_pos = glGetUniformLocation(program, "lightPos")

            # Capturando qualificador de geometria e enviando informação
            loc_geometric = glGetUniformLocation(program, "geometric")
            glUniform1f(loc_geometric, self._geometric)

            # Capturando qualificador de cor e enviando informação
            loc_color = glGetUniformLocation(program, "color")
            glUniform4f(loc_color, 0.0, 1.0, 0.0, 1.0)

            # Define se será mostrado a textura ou não
            glPolygonMode(GL_FRONT_AND_BACK, self._polygonal_mode)
            # Envia as matrizes de transformação
            glUniformMatrix4fv(loc_mat_pre_transl, 1, gl_Draw, np.identity(4)) 
            glUniformMatrix4fv(loc_mat_rot_x, 1, gl_Draw, self._rotation_x()) 
            glUniformMatrix4fv(loc_mat_rot_y, 1, gl_Draw, self._rotation_y()) 
            glUniformMatrix4fv(loc_mat_rot_z, 1, gl_Draw, self._rotation_z()) 
            glUniformMatrix4fv(loc_mat_scale, 1, gl_Draw, self._mat_scale()) 
            glUniformMatrix4fv(loc_mat_transl, 1, gl_Draw, self._mat_translation()) 
            # Envia parâmetros de iluminação
            glUniform1f(loc_ka, self._ka) 
            glUniform1f(loc_kd, self._kd)   
            glUniform1f(loc_ks, self._ks)
            glUniform1f(loc_ns, self._ns)
            # Envia localizacao da luz de iluminação
            if self._light_source:
                glUniform3f(loc_light_pos, self._pos[0], self._pos[1], self._pos[2])
            # Desenha o objeto com primitiva GL_TRIANGLES
            glDrawArrays(GL_TRIANGLES, self._inicial_vert, self._num_vert)

In [801]:
# obj_wave: classe para criação de objetos com textura importados
    # Esta classe é específica para objetos importados de wavefront
    # É filha da classe graphic_element
class obj_wave(graphic_element):
    def __init__(   self, 
                    id_texture, path_obj, path_jpg,
                    inicial_vert = 0, num_vert = 0,
                    pos_x = 0, pos_y = 0, pos_z = 0,
                    angle_x = 0, angle_y = 0, angle_z = 0,
                    scale = 1,
                    linear_speed = 0.05, angular_speed = 0.139, scale_speed = 0.2,
                    limit_sup = 1,
                    limit_inf = -1,
                    ka = 1, kd = 1, ks = 1, ns = 0.10,
                    light_source = False,
                    geometric = False,
                    mag = GL_LINEAR,
                ):
        # Inicializa atributos da mãe
        super().__init__(   inicial_vert = inicial_vert, 
                            num_vert = num_vert,
                            pos_x = pos_x, pos_y = pos_y, pos_z = pos_z,
                            angle_x = angle_x, angle_y = angle_y, angle_z = angle_z,
                            scale = scale,
                            linear_speed = linear_speed, angular_speed = angular_speed, scale_speed = scale_speed,
                            limit_sup = limit_sup,
                            limit_inf = limit_inf,
                            ka = ka, kd = kd, ks = ks, ns = ns,
                            light_source = light_source,
                            geometric = geometric
                        )
        # Atributos de identificação
        self._id_texture = id_texture   # id que identifica qual textura se está trabalhando
        self._path_obj = path_obj       # Caminho do arquivo .obj com malhas
        self._path_jpg = path_jpg       # Caminho do arquivo .jpg com texturas
        # Gerando modelos
        # Carregando os valores das coordenadas de maior e menor valor de cada eixo
        # Carregando valores médios de cada coordenada
        self._model, self._min_coord, self._max_coord, self._average_coord = self._load_model_from_file()
        # Encontra a maior diferença entre as coordenadas do objeto
        self._max_dif = self.find_max_dif()
        # Corrigindo escala para limita entre [-1,1]
        self._correct_scale = 1/(2*self._max_dif)
        self._scale = self._scale * self._correct_scale
        self._scale_speed *= self._correct_scale
        # Variável de magnificação que define se será linear ou nearest
        self._mag = mag
        # Carrega textura
        self._load_texture_from_file()
    
    # Alterna entre os modos de magnificação
    def turn_mag(self):
        if self._mag == GL_NEAREST:
            self._mag = GL_LINEAR
            self._load_texture_from_file()
        else:
            self._mag = GL_NEAREST
            self._load_texture_from_file()

    # Encontra a maior diferença entre as coordenadas do objeto
    def find_max_dif(self):
        max_dif = -1
        for i in range(0,3):
            max_dif = max(max_dif, abs(self._min_coord[i] - self._max_coord[i]))
        return max_dif

    # Calcula a posição final de uma coordenada fornecida
    def _calc_mat_pos_final(self, coord):
        pos_final = np.dot(self._mat_pre_translation().reshape(4,4), coord)
        pos_final = np.dot(self._rotation_z().reshape(4,4), pos_final)
        pos_final = np.dot(self._rotation_y().reshape(4,4), pos_final)
        pos_final = np.dot(self._rotation_x().reshape(4,4), pos_final)
        pos_final = np.dot(self._mat_scale().reshape(4,4), pos_final)
        return np.dot(self._mat_translation().reshape(4,4), pos_final)
    
    # Verifica se com a mudança de um atributo para o novo valor faz com que o objeto saida tela
    # Entrada: novo e antigo valor e flags indicando qual atributo está sendo alterado
    def _verify_limit(self,
                    new_value,
                    old_value,
                    pos = [False, False, False],
                    angle = [False, False, False],
                    scale = False): 
        
        # Atribui o novo valor ao atributo do objeto temporariamente
        if scale: self._scale = new_value         
        for i in range(0,3):
            if pos[i]: self._pos[i] = new_value
            if angle[i]: 
                self._cos[i] = math.cos(new_value)
                self._sin[i] = math.sin(new_value)
        # Calcula a posição máxima e minima
        pos_max_final = self._calc_mat_pos_final(np.array(self._max_coord + [1]).reshape(4,1))
        pos_min_final = self._calc_mat_pos_final(np.array(self._min_coord + [1]).reshape(4,1))
        # Retorna o valor antigo ao atributo do objeto
        if scale: self._scale = old_value   
        for i in range(0,3):
            if pos[i]: self._pos[i] = old_value
            if angle[i]: 
                self._cos[i] = math.cos(new_value)
                self._sin[i] = math.sin(new_value)
        # print(f'Valor max final: {pos_max_final}')
        # print(f'Valor min final: {pos_min_final}')
        # Verifica se a posição do máxima e mínima são aceitáveis
        for i in range(0,3):
            if (   pos_max_final[i] > self._limit_sup
                or pos_max_final[i] < self._limit_inf
                or pos_min_final[i] > self._limit_sup
                or pos_min_final[i] < self._limit_inf):
                # Se não for, retorna falso
                # print(f'Valor max final: {pos_max_final}')
                # print(f'Valor min final: {pos_min_final}')
                return False
        # Se fo, retorna true
        return True

    # Overriding: alterando a forma como o objeto deve se manter ativo
    def set_on(self, id):
        if id == self._id_texture:
            self._on = True
        else:
            self._on = False

    # Função: carrega o arquivo Wavefront
    # Entrada: nome do arquivo
    # Saida: estrutura que armazena o elemento (vertices, textura e faces)
    def _load_model_from_file(self):
        vertices = []
        texture_coords = []
        faces = []
        # Soma das coordenadas para média com intenção de centralizar posição
        sum_coord = [0, 0, 0]
        num_vertices = 0
        # Menor valor de cada eixo
        min_coord = [0x3f3f3f3f,0x3f3f3f3f,0x3f3f3f3f]
        # Maior valor de cada eixo
        max_coord = [-0x3f3f3f3f,-0x3f3f3f3f,-0x3f3f3f3f]

        material = None

        # Abre o arquivo obj (wavefront) para leitura
        for line in open(self._path_obj, "r"): ## para cada linha do arquivo .obj
            # Se for comentário, ignore esta linha e use a próxima
            if line.startswith('#'): continue

            # Quebra a linha por espaço
            values = line.split()
            # Se não há informações na linha, ignore esta linha e use a próxima
            if not values: continue

            # Recupera as informações
            ### Armazena coordenadas dos vertices do elemento no vetor vertices
            if values[0] == 'v':
                vertices.append(values[1:4])
                # Se alguma coordenada é maior que todas as outras
                for i in range(1,4):
                    if float(values[i]) > float(max_coord[i - 1]):
                        max_coord[i - 1] = float(values[i])
                    if float(values[i]) < float(min_coord[i - 1]):
                        min_coord[i - 1] = float(values[i])
                    sum_coord[i - 1] += float(values[i])
                # Somando coordenadas para fazer média
                num_vertices = num_vertices + 1
            ### Armazena coordenadas das texturas no vetor texture_coords
            elif values[0] == 'vt':
                texture_coords.append(values[1:3])
            ### Define o material 
            elif values[0] in ('usemtl', 'usemat'):
                material = values[1]
            ### Armazena informações sobre a construção das faces
            elif values[0] == 'f':
                face = []
                face_texture = []
                # Para cada elemento da linha que define a função
                for bloco in values[1:]:
                    # Separa o elemento em vetor de elementos separando os números que são separados por /
                    positions = bloco.split('/')
                    # Adiciona o primeiro número na face (que representa o número da linha que encontra-se um vértice para da figura)
                    face.append(int(positions[0]))
                    # Se o vetor com elementos separados por / for maior ou igual que dois
                    # Se o segundo número do elemento for maior do que zero
                    if len(positions) >= 2 and len(positions[1]) > 0:
                        # Adicione o segundo número na face de textura (que representa o número da linha que encontra-se um vértice de textura da figura)
                        face_texture.append(int(positions[1]))
                    else:
                        # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                        face_texture.append(0)
                # Após conseguir, provavelmente, os três valores para face, os três valores para textura e o tipo de material, insira na faces
                faces.append((face, face_texture, material))

        model = {}
        model['vertices'] = vertices
        model['texture'] = texture_coords
        model['faces'] = faces

        return model, min_coord, max_coord, sum_coord/np.full(3,num_vertices)
    
    # Função: associa id com a textura
    # Entradas: o id que queremos associar e o caminho do arquivo .jpg
    # Saida: não possui, apenas associa
    def _load_texture_from_file(self):
        #Definindo o id
        glBindTexture(GL_TEXTURE_2D, self._id_texture)
        #Alterando configurações paramétricas de textura
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
        glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, self._mag)
        #Abre imagem
        img = Image.open(self._path_jpg)
        #Captura as dimensões
        img_width = img.size[0]
        img_height = img.size[1]
        #Transforma imagem para um sequência de bytes em formato raw de arquivo
        image_data = img.tobytes("raw", "RGB", 0, -1)
        #Carregando os dados da imagem
        glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

    # Função: retorna uma lista com coordenadas dos vertices do objeto e outra com as da textura
    # Obs: como as listas de vértices só tem funcionalidade no momento em que são enviados para a GPU,
        # não há necessidade de manter dentro da classe
    def get_vertices_textures(self):
        vertices_list = []    
        textures_coord_list = []
        # Para cada um das faces (num_line(v), num_line(vt), material)
        for face in self._model['faces']:
            # Para cada um dos números que representa a linha do vértice
            for vertice_id in face[0]: # Pega o valor a coordenada do vértice
                vertices_list.append( self._model['vertices'][vertice_id-1] )
            # Para cada um dos números que representa a linha da coordenada da textura
            for texture_id in face[1]:  # Pega o valor a coordenada da textura
                textures_coord_list.append( self._model['texture'][texture_id-1] )
        return vertices_list, textures_coord_list
    
    # Retorna matriz de translação inicial para garantir que o objeto inicialize em (0,0,0)
    def _mat_pre_translation(self):
        return np.array([   1.0, 0.0, 0.0, - self._average_coord[0], 
                            0.0, 1.0, 0.0, - self._average_coord[1], 
                            0.0, 0.0, 1.0, - self._average_coord[2], 
                            0.0, 0.0, 0.0,                     1.0], np.float32)
    
    # Overriding: altera a maneira de desenhar levando em conta a textura e a translação inicial
    def draw(self, loc_mat_pre_transl, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, loc_mat_scale, loc_mat_transl,
             loc_ka, loc_kd, loc_ks, loc_ns, loc_light_pos, gl_Draw = GL_TRUE):
        if self._on:
            # Capturando qualificadores de matrizes de transformação
            loc_mat_rot_x = glGetUniformLocation(program, "mat_rot_x")
            loc_mat_rot_y = glGetUniformLocation(program, "mat_rot_y")
            loc_mat_rot_z = glGetUniformLocation(program, "mat_rot_z")
            loc_mat_scale = glGetUniformLocation(program, "mat_scale")
            loc_mat_transl = glGetUniformLocation(program, "mat_transl")
            loc_mat_pre_transl = glGetUniformLocation(program, "mat_pre_transl")

            # Capturado qualificadores de iluminação
            loc_ka = glGetUniformLocation(program, "ka")
            loc_kd = glGetUniformLocation(program, "kd") 
            loc_ks = glGetUniformLocation(program, "ks")
            loc_ns = glGetUniformLocation(program, "ns")
            loc_light_pos = glGetUniformLocation(program, "lightPos")

            # Capturando qualificador de geometria e enviando informação
            loc_geometric = glGetUniformLocation(program, "geometric")
            glUniform1f(loc_geometric, self._geometric)

            # Define se será utilizado texturas ou não
            glPolygonMode(GL_FRONT_AND_BACK, self._polygonal_mode)
            # Exporta matrizes
            glUniformMatrix4fv(loc_mat_pre_transl, 1, gl_Draw, self._mat_pre_translation())
            glUniformMatrix4fv(loc_mat_rot_x, 1, gl_Draw, self._rotation_x()) 
            glUniformMatrix4fv(loc_mat_rot_y, 1, gl_Draw, self._rotation_y()) 
            glUniformMatrix4fv(loc_mat_rot_z, 1, gl_Draw, self._rotation_z()) 
            glUniformMatrix4fv(loc_mat_scale, 1, gl_Draw, self._mat_scale()) 
            glUniformMatrix4fv(loc_mat_transl, 1, gl_Draw, self._mat_translation())
             # Envia parâmetros de iluminação
            glUniform1f(loc_ka, self._ka) 
            glUniform1f(loc_kd, self._kd)   
            glUniform1f(loc_ks, self._ks)
            glUniform1f(loc_ns, self._ns)
            # Envia localizacao da luz de iluminação
            if self._light_source:
                glUniform3f(loc_light_pos, self._pos[0], self._pos[1], self._pos[2])
            # Ativa textura com id
            glBindTexture(GL_TEXTURE_2D, self._id_texture)
            # Desenha o elemento
            glDrawArrays(GL_TRIANGLES, self._inicial_vert, self._num_vert)

### Carregando os modelos

In [802]:
#Ativando texturas 2D
glEnable(GL_TEXTURE_2D)

#Gerando ids
num_textures = 10
textures = glGenTextures(num_textures)

# Cria luz
main_light = graphic_element(pos_x = 50, pos_y = 50, scale = 0.2, ns = 1000, light_source = True)

# Carregando modelos
path_wave = 'objetos_wavefront'
# Vetor com os modelos
objs_wave = []

# Os nomes devem ser iguais a da pasta
names_obj = ['gato', 'dog', 'pug', 'tigre', 'lobo']

# Gato
id_obj = 0
name_obj = 'gato'
path_obj = f'{path_wave}\{name_obj}\{name_obj}.obj'
path_jpg = f'{path_wave}\{name_obj}\{name_obj}.jpg'
cat = obj_wave(id_obj, path_obj, path_jpg, pos_y = -0.12, ka = 0.8, kd = 0.5, ks = 0.9, ns = 32)
objs_wave.append(cat)

# Dog
id_obj = id_obj + 1
name_obj = 'dog'
path_obj = f'{path_wave}\{name_obj}\{name_obj}.obj'
path_jpg = f'{path_wave}\{name_obj}\{name_obj}.jpg'
dog = obj_wave(id_obj, path_obj, path_jpg, pos_y = -0.012, ka = 0.8, kd = 0.8, ks = 0.9, ns = 32)
objs_wave.append(dog)

# Pug
id_obj = id_obj + 1
name_obj = 'pug'
path_obj = f'{path_wave}\{name_obj}\{name_obj}.obj'
path_jpg = f'{path_wave}\{name_obj}\{name_obj}.jpg'
pug = obj_wave(id_obj, path_obj, path_jpg, pos_y = -0.012, ka = 0.8, kd = 1, ks = 0.9, ns = 32)
objs_wave.append(pug)


### Construção dos vetores de dados

In [803]:
# Quantidade total de vértices de objeto a serem utilizados neste programa
total_len_vert_obj = 0
# Quantidade total de vértices de textura a serem utilizados neste programa
total_len_vert_text = 0
# Vértices de objeto a serem utilizados neste programa
vertices_obj_total = []
# Vértices de textura a serem utilizados neste programa
vertices_text_total = []
# Controla os valores que definem a identificação de cada objeto
identification = []

# Para cada objeto:
for obj in objs_wave:
    # Captura os vertices do objeto e de textura
    vertices_list, textures_coord_list = obj.get_vertices_textures()
    # Declara a identificação
    identification.append((total_len_vert_obj, len(vertices_list)))
    # Adiciona o tamanho total dos vertices de objeto e de textura
    total_len_vert_obj += len(vertices_list)
    total_len_vert_text += len(textures_coord_list)
    # Adiciona os vértices de objeto e de textura
    vertices_obj_total += vertices_list
    vertices_text_total += textures_coord_list

# Constrói o solo
ground = graphic_element(
       inicial_vert = total_len_vert_obj, num_vert = 6, ka = 0.5, kd = 0.5, ks = 0.5, ns = 0.10
    )
total_len_vert_obj += 6
level_ground = -0.25
size_ground  = 50
vertices_obj_total += [(-size_ground, level_ground, size_ground), (size_ground, level_ground, size_ground), (size_ground, level_ground, -size_ground), (-size_ground, level_ground, size_ground), (-size_ground, level_ground, -size_ground), (size_ground, level_ground, -size_ground)]


#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(total_len_vert_obj, [("position", np.float32, 3)])
vertices['position'] = vertices_obj_total

#Finaliza a modelagem dos dados de texturas
textures = np.zeros(total_len_vert_text, [("position", np.float32, 2)])
textures['position'] = vertices_text_total

#Setando as identificações de desenho do objeto
for i, obj in enumerate(objs_wave):
    obj.set_identification(identification[i][0], identification[i][1])

### Manipulação dos espaços de dados

In [804]:
#Solicita dois buffers para GPU
buffer = glGenBuffers(2)

# Enviando dados de vértice
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Enviando dados de textura
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)



# Capturando qualificadores view e projection
loc_view = glGetUniformLocation(program, "view")
loc_projection = glGetUniformLocation(program, "projection")

### Eventos de teclado e mouse

In [805]:
# Define um objeto principal para uso na capturação de eventos
main_obj = objs_wave[0]
# Controladores de teclas para evitar erro
press_keys = [True, True]

# Posicao inicial da camera
cameraPos   = glm.vec3(0.0,  0.0,  1.0)
# Vetor responsável para apontar para frente
cameraFront = glm.vec3(0.0,  0.0, -1.0)
# Vetor auxiliar que aponta para cima em relação a camera
cameraUp    = glm.vec3(0.0,  1.0,  0.0)

def key_event(window,key,scancode,action,mods):
    global objs_wave, main_obj, press_keys
    global cameraPos, cameraFront, cameraUp

    # Ativando objetos
    # Determinando id_obj
    # id_obj = key

    # # Ativa o objeto chamado
    # if id_obj >= 48 and id_obj <= 57:
    #     id_obj = id_obj - 48
    #     # Ativa e desativa objeto
    #     for obj in objs_wave:
    #         obj.set_on(id_obj)
    #         if obj.get_on():
    #             main_obj = obj # Apenas por otimização, captura o objeto ativo
    
    # Ações do objeto
    if key == 262: main_obj.rotate('-', 'y')    #Tecla LEFT: rotacionar para esquerda
    if key == 263: main_obj.rotate('+','y')     #Tecla RIGHT: rotacionar para direita
    
    if key == 264: main_obj.rotate('-','x')     #Tecla DOWN: rotacionar para baixo
    if key == 265: main_obj.rotate('+','x')     #Tecla UP: rotacionar para cima

    if key == 333: main_obj.rotate('-','z')     #Tecla -: rotacionar no sentido antihorario
    if key == 334: main_obj.rotate('+','z')     #Tecla +: rotacionar no sentido horario

    if key == 88: main_obj.distort('-')         #Tecla X: diminuir escala
    if key == 90: main_obj.distort('+')         #Tecla Z: aumentar escala

    # if key == 87: main_obj.move('+','y')        #Tecla W: mover para cima
    # if key == 83: main_obj.move('-','y')        #Tecla S: mover para baixo
    # if key == 65: main_obj.move('-','x')        #Tecla A: mover para esquerda
    # if key == 68: main_obj.move('+','x')        #Tecla D: mover para direita

    if key == 80:                               #Tecla P: altera visualização de textura
        if press_keys[0]:
            main_obj.turn_texture()
            press_keys[0] = False
        else:
            press_keys[0] = True

    if key == 86:                               #Tecla V: altera modo de magnificação
        if press_keys[1]:
            main_obj.turn_mag()
            press_keys[1] = False
        else:
            press_keys[1] = True

    # Componentes da câmera
    # Velocidade da camera
    cameraSpeed = 0.1 # 0.01
    # Ir para frente
    if key == 87 and (action==1 or action==2): # Tecla W
        cameraPos += cameraSpeed * cameraFront
    # Ir para trás
    if key == 83 and (action==1 or action==2): # Tecla S
        cameraPos -= cameraSpeed * cameraFront
    # Ir para esquerda
    if key == 65 and (action==1 or action==2): # Tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    # Ir para direita    
    if key == 68 and (action==1 or action==2): # Tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

    # print('[key event] key=',key)
    # print('[key event] scancode=',scancode)
    # print('[key event] action=',action)
    # print('[key event] mods=',mods)
    # print('-------')

# Variáveis auxiliar para captura de cursor
# Flag para definir se eh a primeira vez que o mouse aparece na tela
firstMouse = True
# yaw: rotação no eixo y
yaw = -90.0 
# pitch: rotação no eixo x
pitch = 0.0
# Valores iniciais da última posição do mouse
lastX =  WIDTH_WINDOW/2
lastY =  HEIGHT_WINDOW/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    # Tratando caso de primeira aparição do mouse
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    # Calculos da variação
    xoffset = xpos - lastX
    yoffset = lastY - ypos
    # Atualizando valor da última posição
    lastX = xpos
    lastY = ypos
    # Calculando yam e pitch aproximadamente
    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity
    yaw += xoffset
    pitch += yoffset

    # Evitando que rotação extremas
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    # Fórmulas matemáticas para calcular o novo cameraFront
    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# Define função de evento para teclado
glfw.set_key_callback(window,key_event)
# Define função de evento para cursor
glfw.set_cursor_pos_callback(window, mouse_event)

In [806]:
# Matriz view: posição e orientação da câmera no espaço 3D. Posiciona a cena em relação à câmera.
def view():
    global cameraPos, cameraFront, cameraUp
    # Parãmetros: posição da câmera, direção do target e câmera up. Câmera direita é calculado internamente pela função da biblioteca glm
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
    mat_view = np.array(mat_view)
    return mat_view

# Matriz Projection: transforma o volume de visualização 3D em um espaço 2D, levando em consideração fatores como a distância dos objetos à câmera
def projection():
    # Neste caso, definimos parâmetros estáticos, mas poderiam ser dinâmicos
    fov = glm.radians(45.0)
    aspect = WIDTH_WINDOW/HEIGHT_WINDOW
    near = 0.1
    far = 100.0
    mat_projection = glm.perspective(fov, aspect, near, far)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Exibindo na tela

In [807]:
glfw.show_window(window)

## Loop principal

In [808]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)

# Loop principal
while not glfw.window_should_close(window):
    # Lê eventos
    glfw.poll_events()
    # Limpa buffer de cor e Z-buffer
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    # Define cor como sendo RGB(1.0,1.0,1.0)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    # Percorre por todos os objetos
    for obj in objs_wave:
        # Se ele estiver ativado (apenas para otimização, refaz a checagem)
        if obj.get_on():
            # Desenha o objeto
            obj.draw(loc_mat_pre_transl, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, 
            loc_mat_scale, loc_mat_transl, loc_ka, loc_kd, loc_ks, loc_ns, loc_light_pos)
    
    # Desenha a luz
    main_light.draw(loc_mat_pre_transl, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, 
            loc_mat_scale, loc_mat_transl, loc_ka, loc_kd, loc_ks, loc_ns, loc_light_pos)
    
    # Desenha solo
    ground.draw(loc_mat_pre_transl, loc_mat_rot_x, loc_mat_rot_y, loc_mat_rot_z, 
            loc_mat_scale, loc_mat_transl, loc_ka, loc_kd, loc_ks, loc_ns, loc_light_pos)

    # Atualiza matriz view e projection
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, view())
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, projection())  
    
    # Gerencia troca de dados entre janela e OpenGL
    glfw.swap_buffers(window)

# Encerra
glfw.terminate()